In [1]:
import numpy as np
import os
os.chdir('./5-1/')
import rnn_utils

### RNN forward propagation

In [2]:
def rnn_cell_forward(xt, a_pre, params):

    '''
    input:
        xt - x<t>, (n_x, m)
        a_pre - a<t-1>, (n_a, m)
        params - dict{
            Wax - (n_a, n_x), xt --> at,
            Waa - (n_a, n_a), a_pre --> at,
            ba - (n_a, 1), xt,a_pre --> at,
            Wya - (n_y, n_a), at --> yt_hat,
            by - (n_y, 1), at --> yt_hat
        }
    output:
        at - (n_a, m)
        yt_hat - (n_y, m)
        cache - cache for backpropagation, (at, a_pre, xt, params)
    '''

    Wax = params['Wax']
    Waa = params['Waa']
    Wya = params['Wya']
    ba = params['ba']
    by = params['by']

    at = np.tanh(np.dot(Waa, a_pre) + np.dot(Wax, xt) + ba)
    yt_hat = rnn_utils.softmax(np.dot(Wya, at) + by)
    cache = (at, a_pre, xt, params)

    return at, yt_hat, cache

In [3]:
def rnn_forward(x, a0, params):

    '''
    input:
        x - x, (n_x, m, T) : (features, samples, times)
        a0 - zeros, (n_a, m)
        params - dict{
            Wax - (n_a, n_x), xt --> at,
            Waa - (n_a, n_a), a_pre --> at,
            ba - (n_a, 1), xt,a_pre --> at,
            Wya - (n_y, n_a), at --> yt_hat,
            by - (n_y, 1), at --> yt_hat
        }
    output:
        a - (n_a, m, T), record at every time
        y_hat - (n_y, m, T), record yt_hat every time
        caches - cache for backpropagation, ([(at, a_pre, xt, params),...], x)
    '''

    caches = []
    n_x, m, T = x.shape
    n_y, n_a = params['Wya'].shape
    a = np.zeros((n_a, m, T))
    y_hat = np.zeros((n_y, m, T))
    at = a0

    for t in range(T):
        at, yt_hat, cache = rnn_cell_forward(x[:,:,t], at, params)
        a[:,:,t] = at
        y_hat[:,:,t] = yt_hat
        caches.append(cache)

    caches = (caches, x)
    
    return a, y_hat, caches


### LSTM forward propagation

In [7]:
def lstm_cell_forward(xt, a_pre, c_pre, params):

    '''
    input:
        xt - (n_x, m)
        a_pre - (n_a, m)
        c_pre - (n_a, m)
        params - dict{
            Wf - (n_a, n_a + n_x), a_pre,xt --> gf,
            bf - (n_a, 1), a_pre,xt --> gf,
            Wu - (n_a, n_a + n_x), a_pre,xt --> gu,
            bu - (n_a, 1), a_pre,xt --> gu,
            Wc - (n_a, n_a + n_x), a_pre,xt --> ct_hat,
            bc - (n_a, 1), a_pre,xt --> ct_hat,
            Wo - (n_a, n_a + n_x), a_pre,xt --> go,
            bo - (n_a, 1), a_pre,xt --> go,
            Wy - (n_y, n_a), at --> yt_hat,
            by - (n_y, 1), at --> yt_hat
        }
    output:
        ct - (n_a, m)
        at - (n_a, m)
        yt_hat - (n_y, m)
        cache - (a_pre, c_pre, xt, at, ct, params)
    '''

    Wf = params['Wf']
    bf = params['bf']
    Wu = params['Wu']
    bu = params['bu']
    Wc = params['Wc']
    bc = params['bc']
    Wo = params['Wo']
    bo = params['bo']
    Wy = params['Wy']
    by = params['by']

    n_x, m = xt.shape
    n_y, n_a = Wy.shape

    a_pre_xt = np.concatenate((a_pre, xt), axis=0)

    gf = rnn_utils.sigmoid(np.dot(Wf, a_pre_xt) + bf)
    gu = rnn_utils.sigmoid(np.dot(Wu, a_pre_xt) + bu)
    ct_hat = np.tanh(np.dot(Wc, a_pre_xt) + bc)
    go = rnn_utils.sigmoid(np.dot(Wo, a_pre_xt) + bo)
    ct = c_pre * gf + ct_hat * gu
    at = np.tanh(ct) *go
    yt_hat = rnn_utils.softmax(np.dot(Wy, at) + by)

    cache = (a_pre, c_pre, xt, at, ct, gf, gu, ct_hat, go, params)

    return ct, at, yt_hat, cache

In [8]:
def lstm_forward(x, a0, params):

    '''
    input:
        x - (n_x, m, T)
        a0 - (n_a, m)
        params - dict{
            Wf - (n_a, n_a + n_x), a_pre,xt --> gf,
            bf - (n_a, 1), a_pre,xt --> gf,
            Wu - (n_a, n_a + n_x), a_pre,xt --> gu,
            bu - (n_a, 1), a_pre,xt --> gu,
            Wc - (n_a, n_a + n_x), a_pre,xt --> ct_hat,
            bc - (n_a, 1), a_pre,xt --> ct_hat,
            Wo - (n_a, n_a + n_x), a_pre,xt --> go,
            bo - (n_a, 1), a_pre,xt --> go,
            Wy - (n_y, n_a), at --> yt_hat,
            by - (n_y, 1), at --> yt_hat
        }
    output:
        c - (n_a, m, T)
        a - (n_a, m, T)
        y_hat - (n_y, m, T)
        caches - ([cache], x)
    '''

    caches = []
    n_x, m, T = x.shape
    n_y, n_a = params['Wy'].shape
    a = np.zeros((n_a, m, T))
    c = np.zeros((n_a, m, T))
    y_hat = np.zeros((n_y, m, T))
    at = a0
    ct = np.zeros((n_a, m))

    for t in range(T):
        ct, at, yt_hat, cache = lstm_cell_forward(x[:,:,t], at, ct, params)
        c[:,:,t] = ct
        a[:,:,t] = at
        y_hat[:,:,t] = yt_hat
        caches.append(cache)

    caches = (caches, x)

    return c, a, y, caches


### RNN backpropagation

In [13]:
def rnn_cell_backward(dat, cache):

    '''
    input:
        dat - (n_a, m), dJ/dat
        cache - (at, a_pre, xt, params)
    output:
        grads_t - dict{
            dxt - (n_x, m),
            da_pre - (n_a, m),
            dWaa - (n_a, n_a),
            dWax - (n_a, n_x),
            dba - (n_a, 1)
        }
    '''

    at, a_pre, xt, params = cache
    Wax = params['Wax']
    Waa = params['Waa']
    Wya = params['Wya']
    ba = params['ba']
    by = params['by']
    
    dWax = np.dot(dat * (1 - at**2), xt.T)
    dWaa = np.dot(dat * (1 - at**2), a_pre.T)
    dba = np.sum(dat * (1 - at**2), axis=1, keepdims=True)
    dxt = np.dot(Wax.T, dat * (1 - at**2))
    da_pre = np.dot(Waa.T, dat * (1 - at**2))

    grads_t = {'dWax':dWax, 'dWaa':dWaa, 'dba':dba, 'dxt':dxt, 'da_pre':da_pre}

    return grads_t

In [18]:
def rnn_backward(da, caches):

    '''
    input:
        da - (n_a, m, T), dJ/dy_hat * dy_hat/da, without dat/da_pre
        caches - cache for backpropagation, ([(at, a_pre, xt, params),...], x)
    output:
        grads - dict{
            dx - (n_x, m, T),
            da0 - (n_a, m),
            dWax - (n_a, n_x),
            dWaa - (n_a, n_a),
            dba - (n_a, 1)
        }
    '''

    caches, x = caches
    n_x, m, T = x.shape
    n_a, m, T = da.shape
    
    dx = np.zeros((n_x, m, T))
    dWax = np.zeros((n_a, n_x))
    dWaa = np.zeros((n_a, n_a))
    dba = np.zeros((n_a, m))
    da_pre = np.zeros((n_a, m))

    for t in reversed(range(T)):
        grads_t = rnn_cell_backward(da[:,:,t] + da_pre, caches[t])
        da_pre = grads_t['da_pre']
        dx[:,:,t] = grads_t['dxt']
        dWax += grads_t['dWax']
        dWaa += grads_t['dWaa']
        dba += grads_t['dba']

    da0 = da_pre
    
    grads = {'dx':dx, 'da0':da0, 'dWax':dWax, 'dWaa':dWaa, 'dba':dba}
    
    return grads

### LSTM backward propagation

In [23]:
def lstm_cell_backward(dat, dct, cache):

    '''
    input:
        dat - (n_a, m)
        dct - (n_a, m), ct --> c_pre, without at --> ct
        cache - (a_pre, c_pre, xt, at, ct, gf, gu, ct_hat, go, params)
    output:
        grads_t - dict{
            dxt - (n_x, m),
            dc_pre - (n_a, m),
            da_pre - (n_a, m),
            dWf - (n_a, n_a+n_x),
            dbf - (n_a, 1),
            dWu - (n_a, n_a+n_x),
            dbu - (n_a, 1),
            dWc - (n_a, n_a+n_x),
            dbc - (n_a, 1),
            dWo - (n_a, n_a+n_x),
            dbo - (n_a, 1)
        }
    '''

    a_pre, c_pre, xt, at, ct, gf, gu, ct_hat, go, params = cache
    n_x, m = xt.shape
    n_a, m = at.shape

    dgo_linear = dat * np.tanh(ct) * go * (1-go)    # gate output without sigmoid
    dct_hat_linear = (dct * gu + go * (1 - np.tanh(ct)**2) * gu * dat) * (1 - ct_hat**2)    # without tanh
    dgu_linear = (dct * ct_hat + dat * go * (1 - np.tanh(ct)**2)* ct_hat) * gu * (1-gu)
    dgf_linear = (dct * c_pre + dat * go * (1 - np.tanh(ct)**2) * c_pre) * gf * (1-gf)
    
    a_pre_xt = np.concatenate((a_pre, xt), axis=0)
    dWf = np.dot(dgu_linear, a_pre_xt.T)
    dbf = np.sum(dgf_linear, axis=1, keepdims=True)
    dWu = np.dot(dgu_linear, a_pre_xt.T)
    dbu = np.sum(dgu_linear, axis=1, keepdims=True)
    dWc = np.dot(dct_hat_linear, a_pre_xt.T)
    dbc = np.sum(dct_hat_linear, axis=1, keepdims=True)
    dWo = np.dot(dgo_linear, a_pre_xt.T)
    dbo = np.sum(dgo_linear, axis=1, keepdims=True)

    dc_pre = dct * gf + dat * go * (1 - np.tanh(ct)**2) * gf
    da_pre = np.dot(params['Wf'][:, :n_a].T, dgf_linear) + \
             np.dot(params['Wu'][:, :n_a].T, dgu_linear) + \
             np.dot(params['Wc'][:, :n_a].T, dct_hat_linear) + \
             np.dot(params['Wo'][:, :n_a].T, dgo_linear)
    dxt = np.dot(params['Wf'][:, n_a:].T, dgf_linear) + \
          np.dot(params['Wu'][:, n_a:].T, dgu_linear) + \
          np.dot(params['Wc'][:, n_a:].T, dct_hat_linear) + \
          np.dot(params['Wo'][:, n_a:].T, dgo_linear)

    grads_t = {
        'dxt': dxt,
        'dc_pre': dc_pre,
        'da_pre': da_pre,
        'dWf': dWf,
        'dbf': dbf,
        'dWu': dWu,
        'dbu': dbu,
        'dWc': dWc,
        'dbc': dbc,
        'dWo': dWo,
        'dbo': dbo
    }

    return grads_t


In [24]:
def lstm_backward(da, caches):

    '''
    input:
        da - (n_a, m, T), dy_hat --> da, without dat --> da_pre
        caches - ([cache], x)
    output:
        grads - dict{
            dx - (n_x, m, T),
            da0 - (n_a, m),
            dWf, dWu, dWc, dWo - (n_a, n_a + n_x),
            dbf, dbu, dbc, dbo - (n_a, 1)
        }
    '''

    caches, x = caches
    n_x, m, T = x.shape
    n_a, m, T = da.shape

    dx = np.zeros((n_x, m, T))
    da_pre = np.zeros((n_a, m)) # T+1: da_pre
    dc_pre = np.zeros((n_a, m)) # T+1: dc_pre
    dWf = np.zeros((n_a, n_a + n_x))
    dWu = np.zeros_like(dWf)
    dWc = np.zeros_like(dWf)
    dWo = np.zeros_like(dWf)
    dbf = np.zeros((n_a, 1))
    dbu = np.zeros_like(dbf)
    dbc = np.zeros_like(dbf)
    dbo = np.zeros_like(dbf)
    
    for t in reversed(range(T)):
        grads_t = lstm_cell_backward(da[:,:,t] + da_pre, dc_pre, caches[t])
        da_pre = grads_t['da_pre']
        dc_pre = grads_t['dc_pre']
        dx[:,:,t] = grads_t['dxt']
        dWf += grads_t['dWf']
        dWu += grads_t['dWu']
        dWc += grads_t['dWc']
        dWo += grads_t['dWo']
        dbf += grads_t['dbf']
        dbu += grads_t['dbu']
        dbc += grads_t['dbc']
        dbo += grads_t['dbo']

    da0 = da_pre

    grads = {
        'dx': dx,
        'da0': da0,
        'dWf': dWf,
        'dWu': dWu,
        'dWc': dWc,
        'dWo': dWo,
        'dbf': dbf,
        'dbu': dbu,
        'dbc': dbc,
        'dbo': dbo
    }

    return grads

### character rnn model

In [26]:
import cllm_utils
import random
import time

In [30]:
data = open('dinos.txt', 'r').read()
data = data.lower()
chars = list(set(data))
data_size = len(data)
vocab_size = len(chars)
print(data[:100])
print(chars)
print(f'data_size = {data_size}, vocab_size = {vocab_size}')

aachenosaurus
aardonyx
abdallahsaurus
abelisaurus
abrictosaurus
abrosaurus
abydosaurus
acanthopholis
['i', 'z', 'o', 'g', 'b', 'j', 'k', 't', 'q', 'l', 'c', 's', 'y', 'h', 'e', 'w', 'a', 'r', 'n', 'p', '\n', 'v', 'f', 'd', 'm', 'u', 'x']
data_size = 19909, vocab_size = 27


### dict: one for mapping char to index, another for mapping index to char

In [32]:
char_to_idx = {char: idx for idx, char in enumerate(sorted(chars))}
idx_to_char = {idx: char for idx, char in enumerate(sorted(chars))}
print(char_to_idx)
print(idx_to_char)

{'\n': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


### gradient clip

In [40]:
def clip(grads, maxvalue):

    '''
    input:
        grads - dict of gradients
        maxvalue - threshold of gradients, (-maxcalue, maxvalue)
    output:
        grads - grads after clip
    '''

    for k in grads.keys():
        grads[k] = np.clip(grads[k], -maxvalue, maxvalue)

    return grads

### sample

In [89]:
def sample(params, char_to_idx, seed):

    Waa = params['Waa']
    Wax = params['Wax']
    ba = params['b']
    Wya = params['Wya']
    by = params['by']

    vocab_size = by.shape[0]    # = n_x = n_y
    n_a = Waa.shape[0]
    
    xt = np.zeros((vocab_size, 1))  # initial x1
    a_pre = np.zeros((n_a, 1))  # initial a0
    indices = []    # list of index
    idx = -1    # initial previous index to compare with flag
    count = 0
    flag = char_to_idx['\n']    # index of '\n'

    while idx != flag and count < 50:
        at = np.tanh(np.dot(Waa, a_pre) + np.dot(Wax, xt) + ba)
        z = np.dot(Wya, at) + by
        yt_hat = cllm_utils.softmax(z)  # (vocab_size, 1)

        np.random.seed(seed + count)
        idx = np.random.choice(np.arange(vocab_size), p=yt_hat.flatten())
        indices.append(idx)
        # update xt, a_pre
        xt = np.zeros((vocab_size, 1))
        xt[idx, 0] = 1
        a_pre = at
        count += 1

    if count == 50 and idx != flag:
        indices.append(flag)

    return indices

### SGD
#### notice: in cllm_utils.py, a, x, y_hat --> dict; X, Y --> index list

In [73]:
def optimize(X, Y, a0, params, alpha=0.01):

    '''
    input:
        X - one sample of data, int list, [None, 1,2,3,...]
        Y - one sample of data as X, int list, [1,2,3,...,0]
        a0 - initial a_pre
        params - dict of Waa,Wax,Wya,b,by
        alpha - learning rate
    output:
        loss
        grads - dict of gradients
        params - dict of params updated
        at - last at
    '''

    # forward propagation
    loss, cache = cllm_utils.rnn_forward(X, Y, a0=a0, parameters=params)

    # backward propagation through time
    grads, a = cllm_utils.rnn_backward(X, Y, parameters=params, cache=cache)

    # gradients clips
    grads = clip(grads, 5)

    # update params
    params = cllm_utils.update_parameters(params, grads, alpha)

    return loss, grads, params, a[len(X)-1]

In [81]:
examples = data.split('\n')
print(examples[:10])
print(len(examples))

['aachenosaurus', 'aardonyx', 'abdallahsaurus', 'abelisaurus', 'abrictosaurus', 'abrosaurus', 'abydosaurus', 'acanthopholis', 'achelousaurus', 'acheroraptor']
1536


### train

In [104]:
def model(data, loops=4000, n_a=50, sample_times=7):

    '''
    input:
        data - dataset
        loops - iterate times
        n_a - n_a
        sample_times - times of sample to validate model
    output:
        params
    '''

    vocab_size = len(set(data))
    chars = list(set(data))
    char_to_idx = {char: idx for idx, char in enumerate(sorted(chars))}
    idx_to_char = {idx: char for idx, char in enumerate(sorted(chars))}

    params = cllm_utils.initialize_parameters(n_a, vocab_size, vocab_size)
    a0 = np.zeros((n_a, 1))

    examples = data.split('\n')
    np.random.seed(0)
    np.random.shuffle(examples)

    for i in range(loops):
        index = i % len(examples)
        X = [None] + [char_to_idx[char] for char in examples[index]]
        Y = [char_to_idx[char] for char in examples[index]] + [char_to_idx['\n']]

        loss, grads, params, at = optimize(X, Y, a0, params)
        
        if i % 2000 == 0:
            print(f'No.{i+1} iteration loss: {loss}')
            seed = 0
            for j in range(sample_times):
                indices = sample(params, char_to_idx, seed)
                cllm_utils.print_sample(indices, idx_to_char)
                seed += 1
    
    return params, loss

In [105]:
start = time.time()
params, loss = model(data)
end = time.time()
print(f'total: run {end-start} seconds.')
print(f'Finally, loss: {loss}')

No.1 iteration loss: 39.548881516330404
Nkknzexbw
Kknzexbw
Knzexbw
Nzexbw
Zexbw
Exbw
Xbw
No.2001 iteration loss: 22.938709387824456
Mhgnygtas
Iklyhtas
Klygtas
Myftas
Yesaubus
Ductarahopsaurus
Tarasanrus
total: run 9.10140061378479 seconds.
Finally, loss: 13.802386450251815
